In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Objective
$\newcommand{\mat}[1]{\bar{\bar{#1}}}
 \newcommand{\command}[1]{\text{#1}}$
Convolutions will depend on the shape of the kernel. Kernels can be $(K\times K)$, $(1\times K)$, $(K \times 1)$, $(K)$, $((1))$, $(1)$ or scalar. They are convolved with are $N$-dimensional vectors, where there are $K$ populations and $\sum_{i=1}^K N_i = N$.

For illustration here we set $K = 2$, $N_1 = 500$ and $N_2 = 100$. Let $I_1 \equiv \{1, \dotsc, 500\}$ and $I_2 \equiv \{501, \dotsc, 600\}$.

## 2x2 kernel

$$\begin{gather}
κ = \begin{bmatrix} κ_{11} & κ_{12} \\ κ_{21} & κ_{22} \end{bmatrix} &
\mat{S} = \begin{bmatrix}
\vec{s}_1 \\ \vdots \\ \vec{s}_{500} \\ \vec{s}_{501} \\ \vdots \\ \vec{s}_{600}
\end{bmatrix}
\end{gather}$$

($\mat{S}$ is really a collection of vectors, as each $\vec{s}_i$ may be of different length.)

$$\begin{align}
(κ * \mat{S})(t) &= \begin{bmatrix}
\sum_{i \in I_1} \sum_{s \in \vec{s}_i} κ_{11}(t-s) & \sum_{i \in I_2} \sum_{s \in \vec{s}_i} κ_{12}(t-s) \\
\sum_{i \in I_1} \sum_{s \in \vec{s}_i} κ_{21}(t-s) & \sum_{i \in I_2} \sum_{s \in \vec{s}_i} κ_{22}(t-s)
\end{bmatrix} \\
%
(κ * \mat{S})(t) &= \command{stack}( \,\underbrace{\command{sum}( \underbrace{\,κ_{: j}(t-s)}_{2D \text{ vector}} \; \command{for} \;\vec{s} \,\command{in}\, \mat{S}_{I_j} \;\command{for} \; s \;\command{in} \;\vec{s} )}_{j\text{-th column}} \;\command{for}\; j \; \command{in} \; \command{range}(K) )^\intercal
\end{align}$$

The result of this operation is a $(2\times2)$ matrix.

The pseudocode computes each column in one call and stacks them. Thus the result needs to be transposed to have the expected `[from idx][to idx]` indexing.

## (2,) kernel

$$\begin{gather}
κ = \begin{bmatrix} κ_{1} \\ κ_{2} \end{bmatrix} &
\mat{S} = \begin{bmatrix}
\vec{s}_1 \\ \vdots \\ \vec{s}_{500} \\ \vec{s}_{501} \\ \vdots \\ \vec{s}_{600}
\end{bmatrix}
\end{gather}$$

$$\begin{align}
(κ * \mat{S})(t) &= \begin{bmatrix}
\sum_{s \in \vec{s}_1} κ_{1}(t-s) \\ \vdots \\ 
\sum_{s \in \vec{s}_{500}} κ_{1}(t-s)  \\ 
\sum_{s \in \vec{s}_{501}} κ_{1}(t-s) \\ \vdots \\ 
\sum_{s \in \vec{s}_{600}} κ_{1}(t-s)
\end{bmatrix} \\
%
(κ * \mat{S})(t) &= \command{concatenate}( [\command{stack}( \command{sum}(κ_i(t-s) \;\command{for}\; s \;\command{in}\; \vec{s}_i) \;\command{for}\; i \;\command{in}\; I_j ) \;\command{for}\; j \;\command{in}\; \command{range}(K)] )
\end{align}$$

The result of this operation is an $(N,)$ vector.

# Implementation

In [9]:
def f(Δt, from_idx=slice(None,None)):
    return np.array([[0.1, 1],
                     [-1,  2]])[:, from_idx] * np.exp(-Δt/0.1)

spike_times = [[0.1, 0.5, 1, 4, 8],
               [0.3, 5],
               [4, 9, 9.1, 9.2, 9.3, 9.4],
               [2, 3, 4, 5, 6, 7, 8],
               [3, 8, 12]]
pop_slices = [slice(0,3), slice(3,5)]
t = 9

## 2x2 kernel

In [7]:
np.stack (
     np.sum( f(t-s, from_pop_idx)
             for spike_list in spike_times[pop_slices[from_pop_idx]]
             for s in spike_list )
     for from_pop_idx in range(len(pop_slices)) ).T
    # We don't need to specify an axis in the sum, because the sum is over distinct
    # arrays f(.,.), and so np.sum sums the arrays but doesn't flatten them.

array([[  8.57910703e+00,   1.06864746e+13],
       [ -8.57910703e+01,   2.13729492e+13]])

In [15]:
# Confirmation
Aidcs, A = np.empty((2,2), dtype='object'), np.zeros((2,2))
for i in range(2):
    for j in range(2):
        Aidcs[i,j] = (i+1,j+1)
        A[i,j] = np.sum( f(t-s)[i,j] for spike_list in spike_times[pop_slices[j]] for s in spike_list )
        
print(Aidcs)
print(A)

[[(1, 1) (1, 2)]
 [(2, 1) (2, 2)]]
[[  8.57910703e+00   1.06864746e+13]
 [ -8.57910703e+01   2.13729492e+13]]


## (2,) kernel